In [1]:
import warnings, os
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from glob import glob


import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from aggmap import AggMap, loadmap
import seaborn as sns


import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc


## LGR, RF, KNN, ROF, XGBoost

In [2]:
from sklearn.ensemble import RandomForestClassifier
from rotation_forest import RotationForestClassifier

In [3]:
def score(dfr):
    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])
    auc = roc_auc_score(y_true, y_score)
    ber =  (fp / (fp + tn) + fn / (tp + fn)) * 0.5

    print('acc: %.3f, roc-auc: %.3f, BER: %.3f' % (acc,auc,ber))

    return acc, auc


def get_best_params(X, y):
    clf = xgb.XGBClassifier(eval_metric='logloss', silent=True, verbose=0)
    parameters = {'max_depth': [4, 6, 9],
                  'n_estimators': [20, 50, 100]}

    grid = GridSearchCV(clf, parameters, scoring='roc_auc', n_jobs=9, iid=False, cv=5, verbose=0)
    gres = grid.fit(X, y)
    return grid.best_params_

In [ ]:
lst = glob('/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/*.h5')

n_fold = 5

res = []
for p in lst:
    
    print('#'*50 + ' %s ' % p + '#'*50 )
    dfx = pd.read_hdf(p, key = 'expression')
    dfy = pd.read_hdf(p, key = 'labels')
    print(p, dfy.value_counts().to_dict())

    outer = KFold(n_splits = n_fold, shuffle = True, random_state = 123)
    outer_idx = list(outer.split(dfx, dfy))

    fold_all = []
    for i, idx in enumerate(outer_idx):

        fold_num = "fold_%s" % str(i).zfill(2) 

        train_idx, test_idx = idx

        # subset feature selection by the fold change
        dfxy_train = dfx.iloc[train_idx].join(dfy.iloc[train_idx])
        abds = dfxy_train.groupby(dfy.name).mean()
        fc = abs(abds.iloc[0] - abds.iloc[1])
        selected = fc[fc > 0.5].index

        trainX = dfx[selected].iloc[train_idx].values
        trainY = dfy.iloc[train_idx].values

        testX = dfx[selected].iloc[test_idx].values
        testY = dfy.iloc[test_idx].values

        ## training
        print("\nInput train and test X shape is %s, %s  \n" % (trainX.shape,  testX.shape))
        print("Getting the best parameters by gridsearch \n")
        best_params = get_best_params(trainX, trainY)
        clf = xgb.XGBClassifier(eval_metric='logloss', **best_params, n_jobs=-1)
        clf.fit(trainX, trainY)

        ## evaluation
        y_true = testY
        y_score = clf.predict_proba(testX)[:,1]
        dfr = pd.DataFrame([y_true, y_score]).T
        dfr.columns = ['y_true', 'y_score']
        dfr.index = dfy.iloc[test_idx].index
        acc, auc = score(dfr)
        
        fold_all.append(auc)
        res.append([auc, i, p, len(selected), best_params])
        
    print('The auc score for %s is %s.' % (p, np.mean(fold_all)))

################################################## /home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_THCA_stage.h5 ##################################################
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_THCA_stage.h5 {0: 291, 1: 222}

Input train and test X shape is (410, 173), (103, 173)  

Getting the best parameters by gridsearch 

[22:56:27] WARNING: ../src/learner.cc:576: 
Parameters: { "silent", "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


acc: 0.641, roc-auc: 0.685, BER: 0.374

Input train and test X shape is (410, 137), (103, 137)  

Getting the best parameters by gridsearch 

[23:33:41] WARNING: ../src/learner.cc:576: 
Parameters: { "silent", "verbose" } might not be used.

  

In [6]:
dfres = pd.DataFrame(res, columns = ['score', 'fold', 'task', 'selected_features', 'best_params'])
dfres.to_csv('./FS_XGB_5FCV_results_stage.csv')
dfres.groupby('task').mean().sort_values('score',ascending=False)

,score,fold,selected_features
task,,,
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_KIRC_stage.h5,0.766197,2.0,3130.4
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_THCA_stage.h5,0.701673,2.0,153.6
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_COAD_stage.h5,0.687715,2.0,529.4
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_UCEC_stage.h5,0.671122,2.0,2879.4
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_LIHC_stage.h5,0.655426,2.0,1544.4
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_LUAD_stage.h5,0.642193,2.0,3732.2
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_validate_LUSC_stage.h5,0.609730,2.0,217.0
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_train_SKCM_stage.h5,0.606507,2.0,1588.0
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-S/O_z-score_test_BRCA_stage.h5,0.597326,2.0,304.0
